<h1 style="text-align: center;" markdown="1">TP2: SVM Regressor con tuneo de parametros</h1>
![](../data/icon_properati-data.png)

In [1]:
import numpy as np 
import pandas as pd

# plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

# modules
import knn as knnlibrary

Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
properties =knnlibrary.get_dataset()
properties.describe()

/opt/conda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,3.099760e+05,311335.000000,311335.000000,3.335230e+05,3.335230e+05,3.335230e+05,260093.000000,3.075060e+05,2.186000e+05,2.728530e+05,36414.000000,229854.000000,3.592400e+04
mean,3.563875e+06,-34.633643,-59.313466,3.967361e+05,4.062802e+06,2.420167e+05,292.756133,6.651399e+03,1.918414e+03,5.521272e+03,14.933734,3.093977,4.898150e+03
std,3.919931e+05,2.141045,2.449846,1.436736e+06,7.128929e+06,4.189991e+05,2164.556274,3.606644e+06,3.893250e+03,2.520627e+04,184.897796,1.700742,1.089441e+05
min,3.427208e+06,-54.833014,-122.419415,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-7.000000e+01,0.000000e+00,5.000000e-05,1.000000,1.000000,1.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,6.948895e+06,37.774929,3.379206,6.500000e+08,8.194558e+08,5.000000e+07,200000.000000,2.000000e+09,1.271027e+06,4.000000e+06,27201.000000,40.000000,1.000150e+07


In [3]:
# filtro por CABA y GBA
properties_caba = properties[(properties['place_with_parent_names'].str.contains('Capital Federal') \
                             | properties['place_with_parent_names'].str.contains('Bs.As. G.B.A.'))]

properties_caba = properties_caba[properties_caba['price'].notnull() & properties_caba['price'] > 0 & properties_caba['place_name'].notnull()]

# eliminamos propiedades con mas de 54 pisos
properties_caba = properties_caba[properties_caba['floor'] <= 60]
# eliminamos propiedades con mas de 9 pisos - ver analisis
properties_caba = properties_caba[properties_caba['rooms'] <=  10]
# eliminamos propiedades con mas de 2000 m2 de superficie cubierta - ver analisis
properties_caba = properties_caba[(properties_caba['surface_covered_in_m2'] <= 4000) & (properties_caba['surface_covered_in_m2'] >= 0)]
# eliminamos propiedades con mas de 2000 m2 de superficie cubierta - ver analisis
properties_caba = properties_caba[(properties_caba['surface_total_in_m2']<= 6000)  & (properties_caba['surface_total_in_m2'] >= 0)]
properties_caba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9204 entries, 16 to 34504
Data columns (total 27 columns):
id                            9204 non-null object
created_on                    9204 non-null object
operation                     9204 non-null object
property_type                 9204 non-null object
place_name                    9204 non-null object
place_with_parent_names       9204 non-null object
country_name                  9204 non-null object
state_name                    9204 non-null object
geonames_id                   8045 non-null float64
lat-lon                       8500 non-null object
lat                           8500 non-null float64
lon                           8500 non-null float64
price                         9204 non-null float64
currency                      9204 non-null object
price_aprox_local_currency    9204 non-null float64
price_aprox_usd               9204 non-null float64
surface_total_in_m2           9204 non-null float64
surface_covered_i

In [4]:
# filtro columnas segun lo que hay en el dataset
attributes = ['created_on','property_type','place_name','place_with_parent_names',\
              'country_name','state_name','lat','lon','surface_total_in_m2','surface_covered_in_m2',\
              'floor','rooms', 'price']
properties_caba[attributes].info()
properties_caba_with_price_attributes = properties_caba[attributes]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9204 entries, 16 to 34504
Data columns (total 13 columns):
created_on                 9204 non-null object
property_type              9204 non-null object
place_name                 9204 non-null object
place_with_parent_names    9204 non-null object
country_name               9204 non-null object
state_name                 9204 non-null object
lat                        8500 non-null float64
lon                        8500 non-null float64
surface_total_in_m2        9204 non-null float64
surface_covered_in_m2      9204 non-null float64
floor                      9204 non-null float64
rooms                      9204 non-null float64
price                      9204 non-null float64
dtypes: float64(7), object(6)
memory usage: 1006.7+ KB


In [5]:
from sklearn.preprocessing import LabelEncoder

properties_caba_with_price_attributes.fillna('NaN', inplace=True)
# atributos categoricos
encoder = LabelEncoder()

properties_caba_with_price_attributes['property_type'] = encoder.fit_transform(properties_caba_with_price_attributes[['property_type']])
#properties_caba_with_price_attributes['operation'] = encoder.fit_transform(properties_caba_with_price_attributes[['operation']])
properties_caba_with_price_attributes['place_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['place_name']])
properties_caba_with_price_attributes['place_with_parent_names'] = encoder.fit_transform(properties_caba_with_price_attributes[['place_with_parent_names']])
properties_caba_with_price_attributes['country_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['country_name']])
properties_caba_with_price_attributes['state_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['state_name']])
properties_caba_with_price_attributes['created_on'] = encoder.fit_transform(properties_caba_with_price_attributes[['created_on']])
# completamos valores nan
from sklearn.preprocessing import Imputer
imputer_mean = Imputer(missing_values='NaN', strategy='mean', axis=0)
properties_caba_with_price_attributes["lat"] = imputer_mean.fit_transform(properties_caba_with_price_attributes[["lat"]])
properties_caba_with_price_attributes["lon"] = imputer_mean.fit_transform(properties_caba_with_price_attributes[["lon"]])

properties_caba_with_price_attributes.head()

/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A

,created_on,property_type,place_name,place_with_parent_names,country_name,state_name,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,price
16,133,1,129,69,0,1,-34.680247,-58.553347,59.0,55.0,1.0,2.0,93000.0
17,133,1,129,69,0,1,-34.680247,-58.553347,59.0,55.0,2.0,2.0,93000.0
18,133,1,129,69,0,1,-34.680247,-58.553347,59.0,55.0,3.0,2.0,93000.0
19,133,1,129,69,0,1,-34.680247,-58.553347,59.0,55.0,4.0,2.0,93000.0
20,133,1,129,69,0,1,-34.680247,-58.553347,55.0,55.0,1.0,2.0,93000.0


In [6]:
# separamos el set de train
from sklearn.model_selection import train_test_split
import datetime

now = datetime.datetime.now()

# separamos el train de traing para validarlo luego usando un 30% de los datos
X, y = properties_caba_with_price_attributes.iloc[:, properties_caba_with_price_attributes.columns != 'price'].values, properties_caba_with_price_attributes.iloc[:, properties_caba_with_price_attributes.columns == 'price'].values
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.2, random_state=200)
    
print(y_train)
    
len(X_test), len(X_train)

[[  159500.]
 [ 1598387.]
 [  127000.]
 ..., 
 [   70000.]
 [  170000.]
 [  340000.]]


(1841, 7363)

In [20]:
from sklearn.grid_search import GridSearchCV
from sklearn import svm

C_range = [1000,5000,10000,15000,20000]
gamma_range = [2,3,4,5]
param_test1 = dict(gamma=gamma_range, C=C_range)

svr1 = svm.SVR(C=1000.0, cache_size=800, coef0=0.0, degree=3,gamma=.01, max_iter=-1, 
              shrinking=True, tol=0.001, verbose=False)

gsearch1 = GridSearchCV(estimator=svr1, param_grid = param_test1,iid=False, cv=10)
gsearch1.fit(X_train,y_train.ravel())

gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: -0.07397, std: 0.01502, params: {'gamma': 2, 'C': 1000},
  mean: -0.07411, std: 0.01505, params: {'gamma': 3, 'C': 1000},
  mean: -0.07416, std: 0.01507, params: {'gamma': 4, 'C': 1000},
  mean: -0.07418, std: 0.01507, params: {'gamma': 5, 'C': 1000},
  mean: -0.06779, std: 0.01393, params: {'gamma': 2, 'C': 5000},
  mean: -0.06856, std: 0.01410, params: {'gamma': 3, 'C': 5000},
  mean: -0.06884, std: 0.01417, params: {'gamma': 4, 'C': 5000},
  mean: -0.06895, std: 0.01419, params: {'gamma': 5, 'C': 5000},
  mean: -0.06030, std: 0.01308, params: {'gamma': 2, 'C': 10000},
  mean: -0.06181, std: 0.01332, params: {'gamma': 3, 'C': 10000},
  mean: -0.06236, std: 0.01341, params: {'gamma': 4, 'C': 10000},
  mean: -0.06256, std: 0.01344, params: {'gamma': 5, 'C': 10000},
  mean: -0.05204, std: 0.01261, params: {'gamma': 2, 'C': 15000},
  mean: -0.05434, std: 0.01281, params: {'gamma': 3, 'C': 15000},
  mean: -0.05518, std: 0.01290, params: {'gamma': 4, 'C': 15000},
  mean: -0.05549, 

In [22]:
# a cada dato le restamos la media y lo dividimos por su desviacion standard
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [40]:
# test C and gamma parameters
C_range = [200000,300000]
gamma_range = [0.7,0.75]
param_test2 = dict(gamma=gamma_range, C=C_range)

svr2 = svm.SVR(C=1000.0, cache_size=800, coef0=0.0, degree=3,gamma=.01, max_iter=-1, 
              shrinking=True, tol=0.001, verbose=False)

gsearch2 = GridSearchCV(estimator=svr2, param_grid = param_test2,iid=False, cv=10, scoring='neg_mean_squared_error')
gsearch2.fit(X_train_std,y_train.ravel())

gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -92814760950.90341, std: 41076514822.86119, params: {'gamma': 0.7, 'C': 200000},
  mean: -92295168621.09671, std: 41101388839.40141, params: {'gamma': 0.75, 'C': 200000},
  mean: -89894946909.69229, std: 40949969217.70699, params: {'gamma': 0.7, 'C': 300000},
  mean: -89250869429.97115, std: 40928739649.68256, params: {'gamma': 0.75, 'C': 300000}],
 {'C': 300000, 'gamma': 0.75},
 -89250869429.97115)

In [43]:
# test kernel parameter
param_test3 = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

svr3 = svm.SVR(C=1000.0, cache_size=800, coef0=0.0, degree=3,gamma=.75, max_iter=-1, 
              shrinking=True, tol=0.001, verbose=False)

gsearch3 = GridSearchCV(estimator=svr3, param_grid = param_test3,iid=False, cv=10, scoring='neg_mean_squared_error')
gsearch3.fit(X_train_std,y_train.ravel())

gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

ValueError: X should be a square kernel matrix

## Realizar un trainning con todos los datos del set

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

X_train_full = stdsc.fit_transform(X)

svr = svm.SVR(C=1000.0, cache_size=800, coef0=0.0, degree=3,gamma=.01, max_iter=-1, 
              shrinking=True, tol=0.001, verbose=False)

svr.fit(X_train_full,y.ravel())

# TEST

In [53]:
# leemos set de test
test_df = pd.read_csv('../data/test/properati_dataset_testing_noprice.csv', low_memory=False)
test_df.head()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...


In [54]:
# transformamos atributos categoricos
test_df['place_name'] = encoder.fit_transform(test_df[['place_name']])
test_df['state_name'] = encoder.fit_transform(test_df[['state_name']])
test_df['place_with_parent_names'] = encoder.fit_transform(test_df[['place_with_parent_names']])
test_df['property_type'] = encoder.fit_transform(test_df[['property_type']])
test_df['created_on'] = encoder.fit_transform(test_df[['created_on']])
test_df['country_name'] = encoder.fit_transform(test_df[['country_name']])

X_test_df = test_df[['created_on','property_type','place_name','place_with_parent_names',\
              'country_name','state_name','lat','lon','surface_total_in_m2','surface_covered_in_m2',\
              'floor','rooms']]

# completamos valores nan
from sklearn.preprocessing import Imputer
imputer_mean = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_test_df['floor'] = X_test_df[['floor']].fillna(1)
X_test_df['rooms'] = X_test_df[['rooms']].fillna(1)

X_test_df["surface_total_in_m2"] = imputer_mean.fit_transform(X_test_df[["surface_total_in_m2"]])
X_test_df["surface_covered_in_m2"] = imputer_mean.fit_transform(X_test_df[["surface_covered_in_m2"]])
X_test_df["lat"] = imputer_mean.fit_transform(X_test_df[["lat"]])
X_test_df["lon"] = imputer_mean.fit_transform(X_test_df[["lon"]])

X_test_std_df = stdsc.transform(X_test_df)

#X_test_std_df.info()

/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:17:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 12 columns):
created_on                 14166 non-null int64
property_type              14166 non-null int64
place_name                 14166 non-null int64
place_with_parent_names    14166 non-null int64
country_name               14166 non-null int64
state_name                 14166 non-null int64
lat                        14166 non-null float64
lon                        14166 non-null float64
surface_total_in_m2        14166 non-null float64
surface_covered_in_m2      14166 non-null float64
floor                      14166 non-null float64
rooms                      14166 non-null float64
dtypes: float64(6), int64(6)
memory usage: 1.3 MB


In [55]:
# prediccion del set de test
#knn = KNeighborsRegressor(n_neighbors=best_k, weights='distance', p=best_p)
#knn.fit(X_train_std[:, k_param], y_train)
X_test_df['prediction'] = svr.predict(X_test_std_df)
X_test_df.head(10)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,created_on,property_type,place_name,place_with_parent_names,country_name,state_name,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,prediction
0,23,1,152,219,0,4,-34.610988,-58.363464,0.0,359.471588,1.0,1.0,7.633202e+05
1,24,1,29,180,0,3,-34.629923,-58.465820,0.0,359.471588,1.0,1.0,1.086648e+06
2,0,1,136,210,0,4,-34.589363,-58.412880,53.0,48.000000,1.0,1.0,1.044591e+05
3,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0,2.960344e+05
4,0,1,40,193,0,4,-34.629923,-58.465820,51.0,51.000000,1.0,1.0,2.537855e+05
5,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0,2.960344e+05
6,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0,2.960344e+05
7,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0,2.960344e+05
8,0,1,47,195,0,4,-34.578721,-58.457095,67.0,67.000000,1.0,1.0,1.638969e+05
9,0,1,7,184,0,4,-34.629923,-58.465820,47.0,47.000000,1.0,1.0,2.926145e+05


In [56]:
# guardo el file
output = pd.DataFrame( data={"id":test_df["id"], "price_usd":X_test_df['prediction']} )
output.to_csv( "../data/result/result_gbv02_"+str(now)+".csv", index=False, quoting=3 )